### Part1

In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql.types import StructType

sc = SparkContext(appName="PysparkStreaming")
ssc = StreamingContext(sc, 5)   # Streaming will execute in each 5 seconds

schema = StructType().add("year", "int").add("month", "string") 
    .add("day", "int").add("site", "int").add("sumofsumvolume", "int")
    
df = spark.readStream.option("sep",",").option("header", "false").schema(schema).csv("part1/")
    
df.registerTempTable("tbl_scats_volume")

query = """select region, Total_Traffic_Volume from  (
select s1.region,count(tv.SumOfSumVolume) as Total_Traffic_Volume
from tbl_scats_volume tv
inner join  tbl_scats_sites s1 
on tv.site=s1.siteid
group by s1.region 
)s
order by Total_Traffic_Volume desc"""

df = spark.sql(query)

df.write.format("org.apache.spark.sql.cassandra")\
.options(table="part1", keyspace="assignment3")\
.save(mode="append")

ssc.start()
ssc.awaitTerminationOrTimeout(30)

-------------------------------------------
Time: 2021-01-22 01:13:25
-------------------------------------------

-------------------------------------------
Time: 2021-01-22 01:13:30
-------------------------------------------

-------------------------------------------
Time: 2021-01-22 01:13:35
-------------------------------------------

-------------------------------------------
Time: 2021-01-22 01:13:40
-------------------------------------------

-------------------------------------------
Time: 2021-01-22 01:13:45
-------------------------------------------

-------------------------------------------
Time: 2021-01-22 01:13:50
-------------------------------------------



### Part 2

In [1]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql.types import StructType

sc = SparkContext(appName="PysparkStreaming")
ssc = StreamingContext(sc, 5)   # Streaming will execute in each 5 seconds

schema = StructType().add("siteid", "string").add("site_description_cap", "string") 
    .add("site_description_lower", "string").add("region", "string").add("lat", "string").add("long", "string")
    
df = spark.readStream.option("sep",",").option("header", "false").schema(schema).csv("part2/")
df.registerTempTable("tbl_scats_sites")

schema = StructType().add("end_time", "string").add("region", "string") \
    .add("site", "int").add("detector", "int").add("sum_volume", "double").add("avg_volume", "double") \
    .add("weighted_avg", "double").add("weighted_var", "double").add("weighted_std_dev", "double")

df_scats_detector = spark.readStream.option("sep",",").option("header", "false").schema(schema).csv("scats_detector/")
df_scats_detector.registerTempTable("tbl_scats_volume_202002")

Hour_df = spark.sql("select EXTRACT(Hour from end_time) as Hour, \
Region,cast(Site as int),cast(Detector as int),cast(Sum_volume as int), cast(avg_volume as int) \
from tbl_scats_volume_202002")
Hour_df.createOrReplaceTempView("Refined_df")


Morning_7 = spark.sql("select Region,Site,Hour, Total_Hourly_Traffic_Volume from ( \
select Region,Site, Hour, sum(Sum_Volume) Total_Hourly_Traffic_Volume \
from Refined_df  \
group by Region,Site,Hour \
)Morning \
where Hour >= 6 and Hour<=11")

Evening_7 = spark.sql("select Region,Site,Hour, Total_Hourly_Traffic_Volume from ( \
select Region,Site, Hour, sum(Sum_Volume) Total_Hourly_Traffic_Volume \
from Refined_df  \
group by Region,Site,Hour \
)Morning \
where Hour >= 15 and Hour<=19")

Morning_7.createOrReplaceTempView("Morning_7")
Evening_7.createOrReplaceTempView("Evening_7")

query = """
(select Region , busiest_sites,"Morning" as type from (
select row_number() over(partition by a.Region  order by  a.Total_Hourly_Traffic_Volume desc) row_num, 
a.Region,a.Site as busiest_sites,a.Total_Hourly_Traffic_Volume from Morning_7 a inner join 
(select Region,Site, max(Total_Hourly_Traffic_Volume) as Peak_Volume from Morning_7
where Total_Hourly_Traffic_Volume<>0
group by Region,Site)b 
on a.Site=b.Site and a.Total_Hourly_Traffic_Volume=b.Peak_Volume and a.Region=b.Region
where Total_Hourly_Traffic_Volume<>0 
)s
where row_num in (1,2,3))
union all
(select distinct Region , busiest_sites,"Evening" as type from (
select row_number() over(partition by a.Region  order by  a.Total_Hourly_Traffic_Volume desc) row_num, 
a.Region,a.Site as busiest_sites,a.Total_Hourly_Traffic_Volume from Evening_7 a inner join 
(select Region,Site, max(Total_Hourly_Traffic_Volume) as Peak_Volume from Evening_7
where Total_Hourly_Traffic_Volume<>0
group by Region,Site)b 
on a.Site=b.Site and a.Total_Hourly_Traffic_Volume=b.Peak_Volume and a.Region=b.Region
where Total_Hourly_Traffic_Volume<>0 
)s
where row_num in (1,2,3))
"""

df = spark.sql(query)

df.write.format("org.apache.spark.sql.cassandra")\
.options(table="part2", keyspace="assignment3")\
.save(mode="append")

ssc.start()
ssc.awaitTerminationOrTimeout(30)

-------------------------------------------
Time: 2021-01-22 01:35:00
-------------------------------------------

-------------------------------------------
Time: 2021-01-22 01:35:05
-------------------------------------------

-------------------------------------------
Time: 2021-01-22 01:35:10
-------------------------------------------

-------------------------------------------
Time: 2021-01-22 01:35:15
-------------------------------------------

-------------------------------------------
Time: 2021-01-22 01:35:20
-------------------------------------------

-------------------------------------------
Time: 2021-01-22 01:35:25
-------------------------------------------



False

### Part 3a

In [1]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql.types import StructType

sc = SparkContext(appName="PysparkStreaming")
ssc = StreamingContext(sc, 5)   # Streaming will execute in each 5 seconds
    
schema = StructType().add("year", "int").add("month", "string") 
    .add("day", "int").add("site", "int").add("sumofsumvolume", "int")
    
df = spark.readStream.option("sep",",").option("header", "false").schema(schema).csv("part1/")
    
df.registerTempTable("tbl_scats_volume")


query = """select year, month, total_Volume from (
select year,month, sum(SumOfSumVolume) total_Volume	
from tbl_scats_volume
group by year,month
)s
where year is not null
order by total_Volume desc
"""

df = spark.sql(query)

df.write.format("org.apache.spark.sql.cassandra")\
.options(table="part3a", keyspace="assignment3")\
.save(mode="append")

ssc.start()
ssc.awaitTerminationOrTimeout(30)

-------------------------------------------
Time: 2021-01-22 01:50:30
-------------------------------------------

-------------------------------------------
Time: 2021-01-22 01:50:35
-------------------------------------------

-------------------------------------------
Time: 2021-01-22 01:50:40
-------------------------------------------

-------------------------------------------
Time: 2021-01-22 01:50:45
-------------------------------------------

-------------------------------------------
Time: 2021-01-22 01:50:50
-------------------------------------------

-------------------------------------------
Time: 2021-01-22 01:50:55
-------------------------------------------



False

### Part 3b

In [1]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql.types import StructType

sc = SparkContext(appName="PysparkStreaming")
ssc = StreamingContext(sc, 5)   # Streaming will execute in each 5 seconds
    
schema = StructType().add("year", "int").add("month", "string") 
    .add("day", "int").add("site", "int").add("sumofsumvolume", "int")
    
df = spark.readStream.option("sep",",").option("header", "false").schema(schema).csv("part1/")
    
df.registerTempTable("tbl_scats_volume")


query = """select year, day, avg_Volume from (
select year,day, avg(SumOfSumVolume) avg_Volume	
from tbl_scats_volume
group by year,day
)s
where year is not null
order by avg_Volume desc
"""

df = spark.sql(query)

df.write.format("org.apache.spark.sql.cassandra")\
.options(table="part3b", keyspace="assignment3")\
.save(mode="append")

ssc.start()
ssc.awaitTerminationOrTimeout(30)

-------------------------------------------
Time: 2021-01-22 01:53:15
-------------------------------------------

-------------------------------------------
Time: 2021-01-22 01:53:20
-------------------------------------------

-------------------------------------------
Time: 2021-01-22 01:53:25
-------------------------------------------

-------------------------------------------
Time: 2021-01-22 01:53:30
-------------------------------------------

-------------------------------------------
Time: 2021-01-22 01:53:35
-------------------------------------------

-------------------------------------------
Time: 2021-01-22 01:53:40
-------------------------------------------



False